<a href="https://colab.research.google.com/github/aabchyein/toy_dcx_data_analytics/blob/main/data_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('./total_db.csv')
df[:5]

,Unnamed: 0,writer,date,review
0,0,스타트업 · v*******,작성일\n08.17,왜 그렇게 말씀 하신지 물어봐도 될까요? 저도 디스크로 고생하고 있는데 한약치료를 ...
1,1,새회사 · l*********\n작성자,작성일\n01.06,복근은 그럼 뭐로해야할까\n나 헹레레 조아하는데..
2,2,국민건강보험공단\n· !*********,작성일\n2020.10.10.,터지면 상태가 어때?
3,3,국민건강보험공단\n· !*********,작성일\n2021.03.27.,허리조심...
4,4,국민건강보험공단\n· !*********,작성일\n2022.08.26.,나으셨나요


#### 결측치 처리

In [26]:
# null값 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1803 entries, 0 to 1802
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1803 non-null   int64 
 1   writer      1803 non-null   object
 2   date        1803 non-null   object
 3   review      1801 non-null   object
dtypes: int64(1), object(3)
memory usage: 56.5+ KB


In [28]:
# null값 삭제
df.dropna(inplace=True)

In [29]:
df.isnull().sum(0)

Unnamed: 0    0
writer        0
date          0
review        0
dtype: int64

In [ ]:
# !pip install python-mecab-ko pyLDAvis

In [30]:
from mecab import MeCab
mecab = MeCab()

In [31]:
review_list = list(df['review'])
review_list[:5]

['왜 그렇게 말씀 하신지 물어봐도 될까요? 저도 디스크로 고생하고 있는데 한약치료를 받을지 신경주사를 한 번 더 맞을지 고민되서요.',
 '복근은 그럼 뭐로해야할까\n나 헹레레 조아하는데..',
 '터지면 상태가 어때?',
 '허리조심...',
 '나으셨나요']

In [32]:
# 품사 달기
mecab.pos(review_list[0])

[('왜', 'MAG'),
 ('그렇게', 'MAG'),
 ('말씀', 'NNG'),
 ('하', 'VV'),
 ('신지', 'EP+EC'),
 ('물어봐도', 'VV+EC'),
 ('될까요', 'VV+EF'),
 ('?', 'SF'),
 ('저', 'NP'),
 ('도', 'JX'),
 ('디스크', 'NNG'),
 ('로', 'JKB'),
 ('고생', 'NNG'),
 ('하', 'XSA'),
 ('고', 'EC'),
 ('있', 'VX'),
 ('는데', 'EC'),
 ('한', 'MM'),
 ('약치료', 'NNG'),
 ('를', 'JKO'),
 ('받', 'VV'),
 ('을지', 'EC'),
 ('신경', 'NNG'),
 ('주사', 'NNG'),
 ('를', 'JKO'),
 ('한', 'MM'),
 ('번', 'NNBC'),
 ('더', 'MAG'),
 ('맞', 'VV'),
 ('을지', 'EC'),
 ('고민', 'NNG'),
 ('되', 'XSV'),
 ('서요', 'EP+EF'),
 ('.', 'SF')]

#### 불용어, 한단어 처리
- 불용어, 한단어 사전 이용

In [45]:
df_stopwords = pd.read_excel('./stopword_list.xlsx')
# df_stopwords = pd.read_excel('../../datasets/stopword_list.xlsx')
df_stopwords[:2]

,stopword
0,가까스로
1,가량


In [40]:
# 불용어 사전
stopwords=df_stopwords['stopword'].values

# 불용어 제거
def remove_stopwords(review_list):
  pos_result = mecab.morphs(review_list) # 형태소 분석 결과

  # 불용어 제거된 단어를 저장할 리스트 초기화
  filtered_words= []

  for word in pos_result:
    if len(word) > 1 and word not in stopwords: # 단어가 불용어가 아닌, 한 단어 이상인 경우에만 추가
      filtered_words.append(word)
  return filtered_words

In [41]:
tokenized_reviews = list()
for reviews in review_list:
  reviews_morphs = remove_stopwords(reviews)
  tokenized_reviews.append(' '.join(reviews_morphs))
print(tokenized_reviews)

['그렇게 말씀 신지 물어봐도 될까요 디스크 약치료 을지 신경 주사 을지', '복근 해야 할까 조아', '터지 상태', '허리 조심 ..', '으셨', '메모리 .. 가누 다가', '면서 정기 병원 검진', '운동 운동 운동 헬스 도움', '사바 은데 발레 나빠져서 ㅠㅠ 근력 복근 길러져서 허리 버티 랄까', 'ㅋㅋㅋ 가까운 야지', '터져서 흘러나왔 다는데 .. 수술 해야 된다는 병원 봐야 서울대 정성근 교수 만나', 'ㅇㄷ', '흐미 ㅠㅠ', '아서 봐요', '사진 주행 거리 브레이크 호스 리퍼 구리 스칠', '수술 최대한 신경 단술 꾸준 운동 자세 교정 필요 엄청 고통 스러워 쾌유', '근력 떨어지 거나 대소변 장애 응급 수술 해야 시술 통증 줄이 운동 추천', '마취 투여 한다는 엄청 바늘 .. 근육 직접 .. 그래두 심해져서 수술 보단 나으니까 위험 해도 셔서 괜찮 침침 정밀 검진 나을 은데', '으면 치료 누워', 'ㅋㅋㅋ', '모든 의학 치료 원리 신체 자생력 회복 시키 더라구 신체 건강 장상 돌려놓 으면 질병 스스로 고치 상처 스스로 아물 소린가 의원 다녀 다녀 볼수록 구나 다니 수원 필요 쪽지 근데 의원 가급적 다니 그래야 필요 진료 으니깐', '힌지 제대로 배우 수행 복구 디스크 터지 활로 운동 져서 데드 200 의외로', '그랴 자꾸 스트레칭 허리 자주 그랴', 'ㅠㅠ', '도움 신분 감사', '디스크 심해 도록 유지 어용 필라테스 허리 무리 올바르 자세 으니 근력 키워서 근육 지탱', '감사 다니 의사 선생 골프 디스크 엄청 다고 평생', '터지 그대로 누워서 암것도 저릿저릿 터지 직전 증상 관리 해라', '맥킨지 인가', '어떠 세여', '터지 상체 움직임 ...', '오우 .. 통증 지속 근데 걱정 .. 30 호전 ㅜㅜ', '고마워 ㅠㅠ', '필라테스 신청 해줘요', '염좌 탸니까 움직이 불편 당연 통증 염좌 결과 염좌 생기 원인 따로 고관절 무릎 관절 발목 체크', '강아지 계단 필요', '유튜브', '응급실 한다는 여친 대립 .. 

In [50]:
df['tokenized_review'] = df['review'].apply(remove_stopwords)
df[:2]

,Unnamed: 0,writer,date,review,stopword_review,tokenized_review
0,0,스타트업 · v*******,작성일\n08.17,왜 그렇게 말씀 하신지 물어봐도 될까요? 저도 디스크로 고생하고 있는데 한약치료를 ...,"[그렇게, 말씀, 신지, 물어봐도, 될까요, 디스크, 약치료, 을지, 신경, 주사,...","[그렇게, 말씀, 신지, 물어봐도, 될까요, 디스크, 약치료, 을지, 신경, 주사,..."
1,1,새회사 · l*********\n작성자,작성일\n01.06,복근은 그럼 뭐로해야할까\n나 헹레레 조아하는데..,"[복근, 해야, 할까, 조아]","[복근, 해야, 할까, 조아]"


#### 대체어 처리
- 대체어 사전 이용

In [46]:
df_replace_list = pd.read_excel('./replace_list.xlsx')
#df_replace_list = pd.read('../../../datasets/replace_list.xlsx')
df_replace_list[:2]

,before_replacement,after_replacement
0,S헬스,삼성헬스
1,LG폰,스마트폰


In [48]:
def replace_word(tokenized_reviews) :
  replaced_review = str()
  for i in range(len(df_replace_list['before_replacement'])):
      # 치환할 단어가 있는 경우 데이터 치환
      if df_replace_list['before_replacement'][i] in tokenized_reviews:
        replaced_review = tokenized_reviews.replace(df_replace_list['before_replacement'][i], df_replace_list['after_replacement'][i])
      # 치환할 단어가 없는 경우 그대로 뱉어냄
      else :
        replaced_review = tokenized_reviews
  return replaced_review

In [51]:
df['preprocess_review'] = df['tokenized_review'].apply(replace_word)
df[:2]

AttributeError: ignored